<a href="https://colab.research.google.com/github/hongjai-rhee/public/blob/master/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function; from torch import nn, optim, cuda
from torch.utils import data; from torchvision import datasets, transforms
import torch.nn.functional as F ; import time

# 런타임유형에서 GPU로 설정

In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

Training MNIST Model on cuda


In [0]:
# Training settings
batch_size = 64
EPOCH=5

# 데이터 불러오기

In [0]:

# MNIST Dataset을 텐서로 변환
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())    

# Data Loader (배치 사이즈에 맞도록 데이터 정리)
train_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset,batch_size=batch_size, shuffle=False)


# 모형설정

In [0]:
class Net(nn.Module):

    def __init__(self):    ## 레이어 구조 설정 (뉴론의 수)
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)   ## 인풋레이어        
        self.l2 = nn.Linear(520, 320)   ## 히든 1
        self.l3 = nn.Linear(320, 240)   ## 히든 2
        self.l4 = nn.Linear(240, 120)   ## 히든 3 
        self.l5 = nn.Linear(120, 10)    ## 아웃풋레이어

    def forward(self, x):  ## 연산방법과 활성화함수 설정
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)               ## no softmax here

model = Net()
model.to(device)   ## GPU 지정

criterion = nn.CrossEntropyLoss()    ## 손실함수 지정
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)   ## 최적화모듈 지정


# 학습과 검증 모듈 (그대로 두면 됨)

In [0]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

# 학습 실행

In [7]:

for epoch in range(EPOCH):
      train(epoch)
      test()
     

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.302506
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.307787
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.308123
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.301888
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.306642
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.302096
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.296173
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.301668
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.306900
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.297768
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.305807
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.310205
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.310447
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.304529
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.297220
Train Epoch: 1 | Batch Status: 9600/60